<a href="https://colab.research.google.com/github/Amogh-S-Acharya/Fifth_Sem_ML_Lab/blob/main/Bayesian_desicion_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()
print(iris.keys())
print(iris.data.shape)
print(iris.target.shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
(150, 4)
(150,)


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(105, 4)
(45, 4)
(105,)
(45,)


In [3]:
import numpy as np
from scipy.stats import norm

class BayesianDecisionTheoryClassifier:
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.n_classes_ = len(self.classes_)
        self.n_features_ = X.shape[1]
        self._calculate_priors(y)
        self._calculate_likelihood_params(X, y)

    def _calculate_priors(self, y):
        self.priors_ = np.zeros(self.n_classes_)
        for i, c in enumerate(self.classes_):
            self.priors_[i] = np.sum(y == c) / len(y)

    def _calculate_likelihood_params(self, X, y):
        self.means_ = np.zeros((self.n_classes_, self.n_features_))
        self.variances_ = np.zeros((self.n_classes_, self.n_features_))
        for i, c in enumerate(self.classes_):
            X_c = X[y == c]
            self.means_[i, :] = np.mean(X_c, axis=0)
            self.variances_[i, :] = np.var(X_c, axis=0)

    def predict(self, X):
        predictions = [self._predict_single(x) for x in X]
        return np.array(predictions)

    def _predict_single(self, x):
        posteriors = []
        for i, c in enumerate(self.classes_):
            prior = self.priors_[i]
            likelihood = 1.0
            for j in range(self.n_features_):
                # Use norm.pdf for Gaussian likelihood
                likelihood *= norm.pdf(x[j], loc=self.means_[i, j], scale=np.sqrt(self.variances_[i, j]))
            posterior = likelihood * prior
            posteriors.append(posterior)
        return self.classes_[np.argmax(posteriors)]


In [4]:
model = BayesianDecisionTheoryClassifier()
model.fit(X_train, y_train)

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.9778
Precision: 0.9794
Recall: 0.9778
F1-score: 0.9777
